In [6]:
import itertools
import networkx as nx
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from itertools import chain
import sys
sys.path.append('/home/ziniu.wzn/BayesCard')
from Pgmpy.factors import factor_product
from Pgmpy.models import BayesianModel, JunctionTree
from Pgmpy.inference.EliminationOrder import (
    WeightedMinFill,
    MinNeighbors,
    MinFill,
    MinWeight,
)
from Pgmpy.factors.discrete import TabularCPD

In [7]:
import numpy as np
import pandas as pd
import time
import sys
import copy
from Models.pgmpy_BN import Pgmpy_BN
from Testing.toy_dataset import *

In [32]:
class VariableElimination(object):
    def __init__(self, model, probs=None):
        model.check_model()
        self.model = model
        if probs is not None:
            self.probs = probs
        elif len(self.model.probs) != 0:
            self.probs = model.probs
        else:
            self.probs = dict()

        if isinstance(model, JunctionTree):
            self.variables = set(chain(*model.nodes()))
        else:
            self.variables = model.nodes()

        self.cardinality = {}
        self.factors = defaultdict(list)

        if isinstance(model, BayesianModel):
            self.state_names_map = {}
            for node in model.nodes():
                cpd = model.get_cpds(node)
                if isinstance(cpd, TabularCPD):
                    self.cardinality[node] = cpd.variable_card
                    cpd = cpd.to_factor()
                for var in cpd.scope():
                    self.factors[var].append(cpd)
                self.state_names_map.update(cpd.no_to_name)

        elif isinstance(model, JunctionTree):
            self.cardinality = model.get_cardinality()

            for factor in model.get_factors():
                for var in factor.variables:
                    self.factors[var].append(factor)
        self.root = self.get_root()

    def get_root(self):
        """Returns the network's root node."""

        def find_root(graph, node):
            predecessor = next(self.model.predecessors(node), None)
            if predecessor:
                root = find_root(graph, predecessor)
            else:
                root = node
            return root

        return find_root(self, list(self.model.nodes)[0])

    def steiner_tree(self, nodes):
        """Returns the minimal part of the tree that contains a set of nodes."""
        sub_nodes = set()

        def walk(node, path):
            if len(nodes) == 0:
                return

            if node in nodes:
                sub_nodes.update(path + [node])
                nodes.remove(node)

            for child in self.model.successors(node):
                walk(child, path + [node])

        walk(self.root, [])
        sub_graph = self.model.subgraph(sub_nodes)
        sub_graph.cardinalities = defaultdict(int)
        for node in sub_graph.nodes:
            sub_graph.cardinalities[node] = self.model.cardinalities[node]
        return sub_graph

    def get_probs(self, attribute, values):
        """
        Calculate Pr(attr in values) where values must be a list
        """
        factor = self.probs[attribute]
        values = [factor.get_state_no(attribute, no) for no in values]
        return np.sum(factor.values[values])


    def _get_working_factors(self, variables=[], evidence=None, return_probs=False, reduce=True):
        """
        Uses the evidence given to the query methods to modify the factors before running
        the variable elimination algorithm.
        Parameters
        ----------
        evidence: dict
            Dict of the form {variable: state}
        Returns
        -------
        dict: Modified working factors.
        """

        useful_var = copy.deepcopy(variables)
        if evidence:
            useful_var += list(evidence.keys())
        sub_graph_model = self.steiner_tree(useful_var)
        variables_sub_graph = set(sub_graph_model.nodes)

        working_factors = dict()
        for node in sub_graph_model.nodes:
            working_factors[node] = set()
            for factor in self.factors[node]:
                if set(factor.variables).issubset(variables_sub_graph):
                    working_factors[node].add((factor, None))

        if return_probs:
            probs = dict()
        # Dealing with evidence. Reducing factors over it before VE is run.
        if evidence and reduce:
            for evidence_var in evidence:
                for factor, origin in working_factors[evidence_var]:
                    factor_reduced = factor.reduce(
                        [(evidence_var, evidence[evidence_var])], inplace=False
                    )
                    if return_probs:
                        factor_reduced.normalize()
                        probs[evidence_var] = self.get_probs(evidence_var, evidence[evidence_var])
                    for var in factor_reduced.scope():
                        if var in working_factors:
                            working_factors[var].remove((factor, origin))
                            working_factors[var].add((factor_reduced, evidence_var))
                if type(evidence[evidence_var]) != list:
                    del working_factors[evidence_var]
        if return_probs:
            return working_factors, sub_graph_model, probs
        return working_factors, sub_graph_model

    def _get_elimination_order(
            self, variables=None, evidence=None, model=None, elimination_order="minfill", show_progress=False
    ):
        """
        Deals with all elimination order parameters given to _variable_elimination method
        and returns a list of variables that are to be eliminated
        Parameters
        ----------
        elimination_order: str or list
        Returns
        -------
        list: A list of variables names in the order they need to be eliminated.
        """
        if model is None:
            model = self.model
        if isinstance(model, JunctionTree):
            all_variables = set(chain(*model.nodes()))
        else:
            all_variables = model.nodes()

        if variables is None:
            to_eliminate = set(all_variables)
        else:
            not_evidence_eliminate = []
            if evidence is not None:
                for key in evidence:
                    if type(evidence[key]) != list:
                        not_evidence_eliminate.append(key)
            to_eliminate = (
                    set(all_variables)
                    - set(variables)
                    - set(not_evidence_eliminate)
            )

        # Step 1: If elimination_order is a list, verify it's correct and return.
        if hasattr(elimination_order, "__iter__") and (
                not isinstance(elimination_order, str)
        ):
            if any(
                    var in elimination_order
                    for var in set(variables).union(
                        set(evidence.keys() if evidence else [])
                    )
            ):
                raise ValueError(
                    "Elimination order contains variables which are in"
                    " variables or evidence args"
                )
            else:
                return elimination_order

        # Step 2: If elimination order is None or a Markov model, return a random order.
        elif (elimination_order is None) or (not isinstance(model, BayesianModel)):
            return to_eliminate

        # Step 3: If elimination order is a str, compute the order using the specified heuristic.
        elif isinstance(elimination_order, str) and isinstance(
                model, BayesianModel
        ):
            heuristic_dict = {
                "weightedminfill": WeightedMinFill,
                "minneighbors": MinNeighbors,
                "minweight": MinWeight,
                "minfill": MinFill,
            }
            elimination_order = heuristic_dict[elimination_order.lower()](
                model
            ).get_elimination_order(nodes=to_eliminate, show_progress=show_progress)
            return elimination_order

    def _variable_elimination(
            self,
            variables,
            operation,
            evidence=None,
            elimination_order="minfill",
            joint=True,
            show_progress=False,
    ):
        """
        Implementation of a generalized variable elimination.

        Parameters
        ----------
        variables: list, array-like
            variables that are not to be eliminated.

        operation: str ('marginalize' | 'maximize')
            The operation to do for eliminating the variable.

        evidence: dict
            a dict key, value pair as {var: state_of_var_observed}
            None if no evidence

        elimination_order: str or list (array-like)
            If str: Heuristic to use to find the elimination order.
            If array-like: The elimination order to use.
            If None: A random elimination order is used.
        """
        # Step 1: Deal with the input arguments.
        if isinstance(variables, str):
            raise TypeError("variables must be a list of strings")
        if isinstance(evidence, str):
            raise TypeError("evidence must be a list of strings")

        # Dealing with the case when variables is not provided.
        if not variables:
            all_factors = []
            for factor_li in self.factors.values():
                all_factors.extend(factor_li)
            if joint:
                return factor_product(*set(all_factors))
            else:
                return set(all_factors)

        # Step 2: Prepare data structures to run the algorithm.
        eliminated_variables = set()
        # Get working factors and elimination order
        # tic = time.time()
        working_factors, sub_graph_model = self._get_working_factors(variables, evidence)
        # toc = time.time()
        # print(f"getting working factors takes {toc-tic} secs")
        elimination_order = self._get_elimination_order(
            variables, evidence, sub_graph_model, elimination_order, show_progress=show_progress
        )
        # print(f"getting elimination orders takes {time.time()-toc} secs")
        # Step 3: Run variable elimination
        if show_progress:
            pbar = tqdm(elimination_order)
        else:
            pbar = elimination_order

        for var in pbar:
            #tic = time.time()
            # print(var)
            if show_progress:
                pbar.set_description("Eliminating: {var}".format(var=var))
            # Removing all the factors containing the variables which are
            # eliminated (as all the factors should be considered only once)
            factors = [
                factor
                for factor, _ in working_factors[var]
                if not set(factor.variables).intersection(eliminated_variables)
            ]
            phi = factor_product(*factors)
            phi = getattr(phi, operation)([var], inplace=False)
            del working_factors[var]
            for variable in phi.variables:
                if variable in working_factors:
                    working_factors[variable].add((phi, var))
            eliminated_variables.add(var)
            # print(f"eliminating {var} takes {time.time()-tic} secs")

        # Step 4: Prepare variables to be returned.
        #tic = time.time()
        final_distribution = set()
        for node in working_factors:
            for factor, origin in working_factors[node]:
                if not set(factor.variables).intersection(eliminated_variables):
                    final_distribution.add((factor, origin))
        final_distribution = [factor for factor, _ in final_distribution]
        # print(final_distribution)
        # print(f"the rest takes {time.time()-tic} secs")
        if joint:
            if isinstance(self.model, BayesianModel):
                return factor_product(*final_distribution).normalize(inplace=False)
            else:
                return factor_product(*final_distribution)
        else:
            query_var_factor = {}
            for query_var in variables:
                phi = factor_product(*final_distribution)
                query_var_factor[query_var] = phi.marginalize(
                    list(set(variables) - set([query_var])), inplace=False
                ).normalize(inplace=False)
            return query_var_factor

    def conditional_query(
            self,
            variables,
            evidence=None,
            elimination_order="weightedminfill",
            joint=True,
            show_progress=False,
    ):
        """
        Parameters
        ----------
        variables: list
            list of variables for which you want to compute the probability

        evidence: dict
            a dict key, value pair as {var: state_of_var_observed}
            None if no evidence

        elimination_order: list
            order of variable eliminations (if nothing is provided) order is
            computed automatically

        joint: boolean (default: True)
            If True, returns a Joint Distribution over `variables`.
            If False, returns a dict of distributions over each of the `variables`.
        """
        common_vars = set(evidence if evidence is not None else []).intersection(
            set(variables)
        )
        if common_vars:
            raise ValueError(
                f"Can't have the same variables in both `variables` and `evidence`. Found in both: {common_vars}"
            )

        return self._variable_elimination(
            variables=variables,
            operation="marginalize",
            evidence=evidence,
            elimination_order=elimination_order,
            joint=joint,
            show_progress=show_progress,
        )


    def query(
            self,
            query,
            elimination_order="minfill",
            show_progress=False,
    ):
        """
        An efficient implementation of probabilistic query

        Parameters
        ----------
        query: Q of form {"attr_name": attr_values}
               attr_values must be a list of values.
        elimination_order: str or list (array-like)
            If str: Heuristic to use to find the elimination order.
            If array-like: The elimination order to use.
            If None: A random elimination order is used.
        """
        tic = time.time()
        # Step 1: Prepare data structures to run the algorithm.
        eliminated_variables = set()
        # Get working factors and elimination order
        # tic = time.time()
        working_factors, sub_graph_model = self._get_working_factors(evidence=query, reduce=False)
        # toc = time.time()
        # print(f"getting working factors takes {toc-tic} secs")

        elimination_order_rest = self._get_elimination_order(variables=list(query.keys()),
                                                        model=sub_graph_model,
                                                        elimination_order=elimination_order,
                                                        show_progress=show_progress)

        elimination_order_variable = self._get_elimination_order(variables=list(elimination_order_rest),
                                                             model=sub_graph_model,
                                                             elimination_order=elimination_order,
                                                             show_progress=show_progress)

        elimination_order = elimination_order_rest + elimination_order_variable
        # print(f"getting elimination orders takes {time.time()-toc} secs")
        # Step 3: Run variable elimination
        if show_progress:
            pbar = tqdm(elimination_order)
        else:
            pbar = elimination_order
        print(pbar)
        result = None
        for var in pbar:
            #tic = time.time()
            print(var)
            if show_progress:
                pbar.set_description("Eliminating: {var}".format(var=var))
            # Removing all the factors containing the variables which are
            # eliminated (as all the factors should be considered only once)
            factors = [
                factor
                for factor, _ in working_factors[var]
                if not set(factor.variables).intersection(eliminated_variables)
            ]
            phi = factor_product(*factors)
            #If its a query variable, we record its probability
            if var in query:
                print(time.time()-tic)
                print(factors)
                if result is None:
                    print(f"variable {var} of prob {self.get_probs(var, query[var])}")
                    result = self.get_probs(var, query[var])
                else:
                    marg_var = [attr for attr in list(phi.variables) if attr != var]
                    phi_var = phi.marginalize(marg_var, inplace=False)
                    phi_var.normalize()
                    values = [phi_var.get_state_no(var, no) for no in query[var]]
                    print(f"variable {var} of prob {np.sum(phi_var.values[values])}")
                    result *= np.sum(phi_var.values[values])
                    
                for factor, origin in working_factors[var]:
                    factor_reduced = factor.reduce(
                        [(var, query[var])], inplace=False
                    )
                    for fact_var in factor_reduced.scope():
                        if fact_var in working_factors:
                            working_factors[fact_var].remove((factor, origin))
                            working_factors[fact_var].add((factor_reduced, var))
            phi = phi.marginalize([var], inplace=False)
            del working_factors[var]
            for variable in phi.variables:
                if variable in working_factors:
                    working_factors[variable].add((phi, var))

            eliminated_variables.add(var)
            # print(f"eliminating {var} takes {time.time()-tic} secs")

        return result

In [11]:
import pickle
with open('/home/ziniu.wzn/BN_checkpoints/check_points/Census_chow-liu.pkl', 'rb') as f:
    BN = pickle.load(f)
#with open('check_points/Census_junction.pkl', 'rb') as f:
    #BN_J = pickle.load(f)

In [33]:
ve = VariableElimination(BN.model)
BN.infer_machine = ve

In [27]:
tic = time.time()
q = ve.conditional_query(["dAge"], {"iClass": 0, "iKorean": [0,1], "iMay75880": 0, "iRagechld": [0, 1], "iRrelchld": 0})
print(time.time()-tic)

0.049874305725097656


In [14]:
q.values

array([0.02012446, 0.0700407 , 0.05920385, 0.15525431, 0.50966802,
       0.06280474, 0.12157609, 0.00132783])

In [15]:
ve.probs["dAge"].values

array([0.17949424, 0.16455985, 0.15089992, 0.13475167, 0.13130455,
       0.12725335, 0.09865048, 0.01308595])

In [15]:
q.values[0]

0.9729037908982489

In [34]:
tic = time.time()
q = ve.query({"iAvail": [0], "iClass": [0], "iKorean": [0,1], "iMay75880": [0], "iRagechld": [0, 1], "iRrelchld": [0]})
print(time.time()-tic)
print(q*BN.nrows)

['iRspouse', 'iRelat1', 'iLooking', 'dOccup', 'dIndustry', 'iRlabor', 'iYearwrk', 'dAge', 'iFertil', 'iMilitary', 'iRvetserv', 'iRrelchld', 'iAvail', 'iKorean', 'iRagechld', 'iClass', 'iMay75880']
iRspouse
iRelat1
iLooking
dOccup
dIndustry
iRlabor
iYearwrk
dAge
iFertil
iMilitary
iRvetserv
iRrelchld
0.3726825714111328
[<DiscreteFactor representing phi(iKorean:2, iRagechld:5, iRrelchld:2, iClass:10, iAvail:5, iMay75880:2) at 0x7fb888ad6cd0>]
variable iRrelchld of prob 0.744034967467157
iAvail
0.939202070236206
[<DiscreteFactor representing phi(iKorean:2, iRagechld:5, iClass:10, iAvail:5, iMay75880:2) at 0x7fb8812fe0d0>]
variable iAvail of prob 0.9677559762191935
iKorean
1.2828075885772705
[<DiscreteFactor representing phi(iKorean:2, iRagechld:5, iClass:10, iMay75880:2) at 0x7fb900eeb150>]
variable iKorean of prob 1.0
iRagechld
1.3965649604797363
[<DiscreteFactor representing phi(iRagechld:5, iClass:10, iMay75880:2) at 0x7fb900ef34d0>]
variable iRagechld of prob 0.8713790305029725
iClass


In [23]:
tic = time.time()
print(BN.query_inefficient({"iMay75880": 0,"iClass": 0, "iRagechld": [0, 4], "iKorean": [0,1], "iAvail": 0, "iRrelchld": 0}))
print(time.time()-tic)

decoded query is {'iMay75880': 0, 'iClass': 0, 'iRagechld': [0, 1], 'iKorean': [0, 1], 'iAvail': 0, 'iRrelchld': 0}
querying iMay75880 with n_distinct [1]
conditioning on ['iClass', 'iRagechld', 'iKorean', 'iAvail', 'iRrelchld'] with probability 0.9893718709242222
querying iClass with n_distinct [1]
conditioning on ['iRagechld', 'iKorean', 'iAvail', 'iRrelchld'] with probability 0.24407894523123658
querying iRagechld with n_distinct [1, 1]
conditioning on ['iKorean', 'iAvail', 'iRrelchld'] with probability 0.8361923356819273
querying iKorean with n_distinct [1, 1]
conditioning on ['iAvail', 'iRrelchld'] with probability 1.0
querying iAvail with n_distinct [1]
conditioning on ['iRrelchld'] with probability 0.9609798873106272
querying iRrelchld with n_distinct [1]
conditioning on [] with probability 0.744034967467157
354924.0
0.19004273414611816


In [26]:
from Evaluation.cardinality_estimation import parse_query_single_table
from time import perf_counter
def evaluate_cardinality(BN, query_path):
    # read all queries
    with open(query_path) as f:
        queries = f.readlines()
    latencies = []
    q_errors = []
    for query_no, query_str in enumerate(queries):
        cardinality_true = int(query_str.split("||")[-1])
        query_str = query_str.split("||")[0]
        print(f"Predicting cardinality for query {query_no}: {query_str}")
        
        query = parse_query_single_table(query_str.strip(), BN)
        card_start_t = perf_counter()
        print(query)
        try:
            cardinality_predict = BN.query_inefficient(query)
        except:
            continue
        if cardinality_predict is None:
            continue
        card_end_t = perf_counter()
        latency_ms = (card_end_t - card_start_t) * 1000
        if cardinality_predict == 0 and cardinality_true == 0:
            q_error = 1.0
        elif cardinality_predict == 0:
            cardinality_predict = 1
        elif cardinality_true == 0:
            cardinality_true = 1

        q_error = max(cardinality_predict / cardinality_true, cardinality_true / cardinality_predict)
        print(f"latency: {latency_ms} and error: {q_error}")
        latencies.append(latency_ms)
        q_errors.append(q_error)
    return latencies, q_errors

In [27]:
query_path = "/home/ziniu.wzn/Census/cardinality/query_one_side.sql"
latencies, q_errors = evaluate_cardinality(BN, query_path)

Predicting cardinality for query 0: SELECT COUNT(*) FROM climate WHERE iAvail = 0 AND iClass = 0 AND iKorean >= 0 AND iKorean <= 1 AND iMay75880 = 0 AND iRagechld >= 0 AND iRagechld <= 4 AND iRrelchld = 0
{'iAvail': [0], 'iClass': [0], 'iKorean': [0, 1], 'iMay75880': [0], 'iRagechld': [4, 2, 0, 3, 1], 'iRrelchld': [0]}
decoded query is {'iAvail': [0], 'iClass': [0], 'iKorean': [0, 1], 'iMay75880': [0], 'iRagechld': [1, 2, 0, 4, 3], 'iRrelchld': [0]}
querying iAvail with n_distinct [1]
conditioning on ['iClass', 'iKorean', 'iMay75880', 'iRagechld', 'iRrelchld'] with probability 0.9727116891644684
querying iClass with n_distinct [1]
conditioning on ['iKorean', 'iMay75880', 'iRagechld', 'iRrelchld'] with probability 0.23979674725158945
querying iKorean with n_distinct [1, 1]
conditioning on ['iMay75880', 'iRagechld', 'iRrelchld'] with probability 1.0
querying iMay75880 with n_distinct [1]
conditioning on ['iRagechld', 'iRrelchld'] with probability 0.9866439050576342
querying iRagechld wit

querying iImmigr with n_distinct [1]
conditioning on ['dIncome4', 'dIndustry', 'iMay75880', 'iRagechld', 'iRemplpar', 'iRiders', 'iRspouse', 'iSchool', 'iVietnam'] with probability 0.9176512532138563
querying dIncome4 with n_distinct [1, 1]
conditioning on ['dIndustry', 'iMay75880', 'iRagechld', 'iRemplpar', 'iRiders', 'iRspouse', 'iSchool', 'iVietnam'] with probability 1.0
querying dIndustry with n_distinct [1, 1, 1]
conditioning on ['iMay75880', 'iRagechld', 'iRemplpar', 'iRiders', 'iRspouse', 'iSchool', 'iVietnam'] with probability 0.20688132492351485
querying iMay75880 with n_distinct [1]
conditioning on ['iRagechld', 'iRemplpar', 'iRiders', 'iRspouse', 'iSchool', 'iVietnam'] with probability 0.9918647661171562
querying iRagechld with n_distinct [1]
conditioning on ['iRemplpar', 'iRiders', 'iRspouse', 'iSchool', 'iVietnam'] with probability 0.35155477695831555
querying iRemplpar with n_distinct [1]
conditioning on ['iRiders', 'iRspouse', 'iSchool', 'iVietnam'] with probability 0.86

querying dTravtime with n_distinct [1]
conditioning on [] with probability 0.5596328334590985
latency: 202.70968973636627 and error: 1.1316376644710844
Predicting cardinality for query 12: SELECT COUNT(*) FROM climate WHERE iCitizen >= 0 AND iCitizen <= 1 AND dHispanic = 0 AND iOthrserv = 0 AND iRelat2 = 0 AND iRemplpar >= 0 AND iRemplpar <= 111 AND iRrelchld = 0 AND iSchool = 1 AND iTmpabsnt = 0 AND dTravtime >= 2 AND dTravtime <= 3
{'iCitizen': [0, 1], 'dHispanic': [0], 'iOthrserv': [0], 'iRelat2': [0], 'iRemplpar': [0, 111], 'iRrelchld': [0], 'iSchool': [1], 'iTmpabsnt': [0], 'dTravtime': [2, 3]}
decoded query is {'iCitizen': [0, 4], 'dHispanic': [0], 'iOthrserv': [0], 'iRelat2': [0], 'iRemplpar': [0, 1], 'iRrelchld': [0], 'iSchool': [0], 'iTmpabsnt': [0], 'dTravtime': [5, 4]}
querying iCitizen with n_distinct [1, 1]
conditioning on ['dHispanic', 'iOthrserv', 'iRelat2', 'iRemplpar', 'iRrelchld', 'iSchool', 'iTmpabsnt', 'dTravtime'] with probability 0.9294565465269262
querying dHispa

querying dIncome1 with n_distinct [1, 1, 1, 1]
conditioning on ['dIncome2', 'dIncome4', 'iMeans', 'iSept80', 'iSubfam1', 'iSubfam2'] with probability 0.9837434450424288
querying dIncome2 with n_distinct [1, 1]
conditioning on ['dIncome4', 'iMeans', 'iSept80', 'iSubfam1', 'iSubfam2'] with probability 1.0
querying dIncome4 with n_distinct [1]
conditioning on ['iMeans', 'iSept80', 'iSubfam1', 'iSubfam2'] with probability 0.6986271627545829
querying iMeans with n_distinct [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
conditioning on ['iSept80', 'iSubfam1', 'iSubfam2'] with probability 0.43820315758546674
querying iSept80 with n_distinct [1]
conditioning on ['iSubfam1', 'iSubfam2'] with probability 0.9837925236531742
querying iSubfam1 with n_distinct [1]
conditioning on ['iSubfam2'] with probability 1.0
querying iSubfam2 with n_distinct [1]
conditioning on [] with probability 0.9737300597774464
latency: 367.68049001693726 and error: 1.0285513258339514
Predicting cardinality for query 19: SELECT COUNT(*) F

querying iImmigr with n_distinct [1]
conditioning on ['dIncome2', 'dIncome6', 'iRagechld', 'dRearning', 'iRlabor'] with probability 0.9164588546624775
querying dIncome2 with n_distinct [1]
conditioning on ['dIncome6', 'iRagechld', 'dRearning', 'iRlabor'] with probability 0.9394030077317072
querying dIncome6 with n_distinct [1]
conditioning on ['iRagechld', 'dRearning', 'iRlabor'] with probability 0.9578996871822353
querying iRagechld with n_distinct [1, 1]
conditioning on ['dRearning', 'iRlabor'] with probability 0.4455178526304385
querying dRearning with n_distinct [1, 1, 1, 1]
conditioning on ['iRlabor'] with probability 0.8387471480558331
querying iRlabor with n_distinct [1, 1, 1, 1, 1, 1]
conditioning on [] with probability 0.7663025239140296
latency: 757.0892609655857 and error: 1.091854947933017
Predicting cardinality for query 24: SELECT COUNT(*) FROM climate WHERE iImmigr >= 0 AND iImmigr <= 1 AND dIncome6 = 0 AND dIncome7 = 0 AND iMarital >= 0 AND iMarital <= 4 AND iRownchld =

...

querying iSept80 with n_distinct [1, 1]
conditioning on ['iVietnam'] with probability 1.0
querying iVietnam with n_distinct [1]
conditioning on [] with probability 0.9659766869992699
latency: 114.89835381507874 and error: 1.007857830965227
Predicting cardinality for query 441: SELECT COUNT(*) FROM climate WHERE dAge >= 3 AND dAge <= 7 AND dIncome3 = 0 AND iMeans = 0 AND iMobility = 1 AND iOthrserv = 0 AND iSex = 1 AND iWork89 >= 0 AND iWork89 <= 1
{'dAge': [5, 6, 3, 4, 7], 'dIncome3': [0], 'iMeans': [0], 'iMobility': [1], 'iOthrserv': [0], 'iSex': [1], 'iWork89': [1, 0]}
decoded query is {'dAge': [5, 3, 2, 1, 4], 'dIncome3': [0], 'iMeans': [0], 'iMobility': [0], 'iOthrserv': [0], 'iSex': [0], 'iWork89': [0, 2]}
querying dAge with n_distinct [1, 1, 1, 1, 1]
conditioning on ['dIncome3', 'iMeans', 'iMobility', 'iOthrserv', 'iSex', 'iWork89'] with probability 0.4315019274054154
querying dIncome3 with n_distinct [1]
conditioning on ['iMeans', 'iMobility', 'iOthrserv', 'iSex', 'iWork89'] wit

querying iCitizen with n_distinct [1]
conditioning on ['iEnglish', 'dHispanic', 'iImmigr', 'dIncome2', 'dIncome4', 'dIncome8', 'iKorean', 'iMarital', 'iPerscare', 'iRvetserv'] with probability 1.0
querying iEnglish with n_distinct [1, 1]
conditioning on ['dHispanic', 'iImmigr', 'dIncome2', 'dIncome4', 'dIncome8', 'iKorean', 'iMarital', 'iPerscare', 'iRvetserv'] with probability 0.9648670321188874
querying dHispanic with n_distinct [1]
conditioning on ['iImmigr', 'dIncome2', 'dIncome4', 'dIncome8', 'iKorean', 'iMarital', 'iPerscare', 'iRvetserv'] with probability 0.9786887608904611
querying iImmigr with n_distinct [1]
conditioning on ['dIncome2', 'dIncome4', 'dIncome8', 'iKorean', 'iMarital', 'iPerscare', 'iRvetserv'] with probability 0.9174539000239977
querying dIncome2 with n_distinct [1, 1]
conditioning on ['dIncome4', 'dIncome8', 'iKorean', 'iMarital', 'iPerscare', 'iRvetserv'] with probability 1.0
querying dIncome4 with n_distinct [1]
conditioning on ['dIncome8', 'iKorean', 'iMarit

querying iCitizen with n_distinct [1]
conditioning on ['iFertil', 'dIncome2', 'dIncome7', 'iSept80'] with probability 0.9145619288192053
querying iFertil with n_distinct [1, 1, 1]
conditioning on ['dIncome2', 'dIncome7', 'iSept80'] with probability 0.27664652491461017
querying dIncome2 with n_distinct [1]
conditioning on ['dIncome7', 'iSept80'] with probability 0.950264458932765
querying dIncome7 with n_distinct [1]
conditioning on ['iSept80'] with probability 0.9337696304981261
querying iSept80 with n_distinct [1]
conditioning on [] with probability 0.9839863970206871
latency: 204.9524448812008 and error: 1.0310852491920708
Predicting cardinality for query 453: SELECT COUNT(*) FROM climate WHERE iFertil = 0 AND dHours >= 0 AND dHours <= 1 AND dIncome2 = 0 AND iLooking = 0 AND dPwgt1 >= 0 AND dPwgt1 <= 1 AND dRpincome >= 2 AND dRpincome <= 4 AND iRPOB = 10 AND iRrelchld = 0 AND iSex >= 0 AND iSex <= 1
{'iFertil': [0], 'dHours': [1, 0], 'dIncome2': [0], 'iLooking': [0], 'dPwgt1': [1, 0]

querying iLang1 with n_distinct [1]
conditioning on ['iRemplpar', 'iSchool', 'dWeek89'] with probability 0.8647300029002336
querying iRemplpar with n_distinct [1]
conditioning on ['iSchool', 'dWeek89'] with probability 0.7249650756244973
querying iSchool with n_distinct [1, 1]
conditioning on ['dWeek89'] with probability 0.8930686707597584
querying dWeek89 with n_distinct [1, 1]
conditioning on [] with probability 0.7008666611072354
latency: 177.03517526388168 and error: 1.084871309977884
Predicting cardinality for query 458: SELECT COUNT(*) FROM climate WHERE iFertil >= 0 AND iFertil <= 4 AND dHour89 = 3 AND iImmigr = 0 AND iLang1 = 2 AND dOccup >= 0 AND dOccup <= 2 AND iRemplpar = 0 AND dTravtime >= 0 AND dTravtime <= 5
{'iFertil': [1, 3, 0, 4, 2], 'dHour89': [3], 'iImmigr': [0], 'iLang1': [2], 'dOccup': [2, 0, 1], 'iRemplpar': [0], 'dTravtime': [5, 1, 2, 0, 4, 3]}
decoded query is {'iFertil': [1, 2, 0, 4, 3], 'dHour89': [1], 'iImmigr': [0], 'iLang1': [0], 'dOccup': [1, 0, 2], 'iRemp

querying iDisabl2 with n_distinct [1]
conditioning on ['iEnglish', 'iRagechld', 'iRelat1', 'iRemplpar', 'iSept80'] with probability 0.8967800103758186
querying iEnglish with n_distinct [1]
conditioning on ['iRagechld', 'iRelat1', 'iRemplpar', 'iSept80'] with probability 0.8777540524902159
querying iRagechld with n_distinct [1]
conditioning on ['iRelat1', 'iRemplpar', 'iSept80'] with probability 0.3732463243576128
querying iRelat1 with n_distinct [1, 1]
conditioning on ['iRemplpar', 'iSept80'] with probability 0.7575181809855289
querying iRemplpar with n_distinct [1]
conditioning on ['iSept80'] with probability 0.7463681602633431
querying iSept80 with n_distinct [1]
conditioning on [] with probability 0.9839863970206871
latency: 199.97680187225342 and error: 1.273479322367979
Predicting cardinality for query 464: SELECT COUNT(*) FROM climate WHERE iCitizen = 0 AND iDisabl1 = 0 AND iFeb55 = 0 AND iImmigr = 0 AND iMobillim >= 0 AND iMobillim <= 2 AND iRownchld = 0 AND iRspouse >= 1 AND iR

In [28]:
for i in [50,90,95,99,100]:
    print(np.percentile(q_errors, i))

1.0615151087683672
1.4680862092599711
2.052275981274598
20.62931050656592
235.0


In [ ]:
from numba import cuda, jit, guvectorize
from numba import int32, float32    # import the types
from numba.experimental import jitclass

spec = [
    ('value', int32),               # a simple scalar field
    ('array', float32[:]),          # an array field
]

@guvectorize(["void(float64[:,:], float64[:,:], float64[:,:])"],
             "(m,n),(n,p)->(m,p)")
def matmul(A, B, result):
    result = A.dot(B)

In [ ]:
a = np.random.randint(10, size=(10,10))
b = np.random.randint(10, size=(10,10))
c = np.ones((10,10))

In [ ]:
tic = time.time()
a.dot(b)
print(time.time()-tic)

In [ ]:
tic = time.time()
matmul(a, b, c)
print(time.time()-tic)

In [ ]:
@jit(target = 'cpu')
def func2(a):
    for i in range(10000000):
        a += 1

In [ ]:
tic = time.time()
func2(1)
print(time.time()-tic)

In [ ]:
def func3(a):
    for i in range(10000000):
        a += 1

In [ ]:
tic = time.time()
func3(1)
print(time.time()-tic)

In [ ]:
@jit
def reduce2(phi_o, values, inplace=True):
    """
    Reduces the factor to the context of given variable values.

    Parameters
    ----------
    values: list, array-like
        A list of tuples of the form (variable_name, variable_state).

    inplace: boolean
        If inplace=True it will modify the factor itself, else would return
        a new factor.

    Returns
    -------
    DiscreteFactor or None: if inplace=True (default) returns None
                    if inplace=False returns a new `DiscreteFactor` instance.
    """
    # Check if values is an array
    if isinstance(values, str):
        raise TypeError("values: Expected type list or array-like, got type str")

    if not all([isinstance(state_tuple, tuple) for state_tuple in values]):
        raise TypeError(
            "values: Expected type list of tuples, get type {type}", type(values[0])
        )

    # Check if all variables in values are in the factor
    #for var, _ in values:
     #   if var not in self.variables:
      #      raise ValueError(f"The variable: {var} is not in the factor")

    phi = phi_o if inplace else phi_o.copy()

    # Convert the state names to state number. If state name not found treat them as
    # state numbers.
    for i, (var, state_name) in enumerate(values):
        if type(state_name) == list:
            values[i] = (var, [phi.get_state_no(var, no) for no in state_name])
        else:
            values[i] = (var, phi.get_state_no(var, state_name))

    var_index_to_del = []
    slice_ = [slice(None)] * len(phi.variables)
    point_query = True
    cardinality = phi.cardinality
    state_names = phi.state_names
    del_state_names = []
    for var, state in values:
        var_index = phi.variables.index(var)
        slice_[var_index] = state
        if type(state) == list:
            point_query = False
            cardinality[var_index] = len(state)
            state_names[var] = state
        else:
            var_index_to_del.append(var_index)
            del_state_names.append(var)

    var_index_to_keep = sorted(
        set(range(len(phi.variables))) - set(var_index_to_del)
    )
    # set difference is not gaurenteed to maintain ordering
    phi.variables = [phi.variables[index] for index in var_index_to_keep]
    phi.cardinality = cardinality[var_index_to_keep]
    phi.del_state_names(del_state_names)
    phi.values = phi.values[tuple(slice_)]

    if not point_query:
        super(DiscreteFactor, phi).store_state_names(
            phi.variables, cardinality, state_names
        )

    if not inplace:
        return phi

In [ ]:
f = list(f['iRelat1'])[0][0]

In [ ]:
f

In [ ]:
s = [("iRspouse", [1])]

In [ ]:
print(a[0][0].values)
print(a[1][0].values)
print(b.values)

In [ ]:
tic = time.time()
a = f.reduce(s, inplace=False)
print(time.time()-tic)

In [ ]:
a.values.shape
a.cardinality

In [ ]:
np.sum(a.values)

In [ ]:
tic = time.time()
reduce2(f, s, inplace=False)
print(time.time()-tic)

In [ ]:
tic = time.time()
f, s =ve._get_working_factors(["iAvail"], {"iClass": 0, "iKorean": [0,1], "iMay75880": 0, "iRagechld": [0, 4], "iRrelchld": 0})
print(time.time() - tic)

In [ ]:
def get_probs(ve, attribute, values):
    factor = ve.probs[attribute]
    values = [factor.get_state_no(attribute, no) for no in values]
    return np.sum(factor.values[values])

In [ ]:
get_probs(ve, "iClass", [1,2,3])